# Simple RAG

Retrieval-Augmented Generation(RAG)은 정보 검색과 생성형 언어 모델을 결합한 접근 방식입니다. 이 방식은 외부 지식을 모델에 활용할 수 있게 해줘서, 더 정확하고 사실 기반의 답변을 생성하는 데 도움이 됩니다.


#### 이 노트북의 주요 단계는 다음과 같습니다:
1. **데이터 수집 및 전처리**: 텍스트 데이터를 불러오고, 사용할 수 있는 형태로 정리합니다.
2. **청킹(Chunking)**: 검색 효율을 높이기 위해 데이터를 적당한 크기의 조각으로 나눕니다.
3. **임베딩 생성**: 조각낸 텍스트를 임베딩 모델을 통해 숫자 벡터로 바꿔줍니다.
4. **Semantic Search**: 사용자의 질문과 관련 있는 텍스트 조각을 의미 기반으로 찾아냅니다.
5. **응답 생성**: 검색된 내용을 바탕으로 언어 모델이 최종 답변을 만들어냅니다.

## 환경 설정하기
필요한 라이브러리를 가져옵니다.

In [ ]:
import os
import numpy as np
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

API_KEY = os.environ.get('api_key')
gemini_API_KEY = os.environ.get('gemini_api_key')

## OpenAI API 클라이언트 설정하기

In [ ]:
from openai import OpenAI

client_openai = OpenAI(api_key = API_KEY)

## PDF 파일에서 텍스트 추출하기
RAG를 구현하려면 먼저 텍스트 데이터 소스가 필요합니다. 저는 gemini를 이용해 pdf에서 텍스트를 추출하는 방식을 사용합니다.  
만약 txt 형태로 파일이 존재한다면 `load_text_file` 함수를 사용하면됩니다.

In [2]:
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    # API 키 설정
    genai.configure(api_key=gemini_API_KEY)
    client = genai.GenerativeModel('gemini-2.0-flash-lite')

    # PDF 파일 업로드
    with open(pdf_path, "rb") as file:
        file_data = file.read()


    prompt = "Extract all text from the provided PDF file."
    response = client.generate_content([
        {"mime_type": "application/pdf", "data": file_data},
        prompt
    ],generation_config={
            "max_output_tokens": 8192  # 최대 출력 토큰 수 설정 (예: 8192 토큰, 약 24,000~32,000자)
    })
    return response.text

In [3]:
# 이미 text 파일로 저장되어 있다면 load_text_file 함수를 사용하면 됩니다.
def load_text_file(pdf_path):

    # text 파일 로드
    with open(pdf_path, "r", encoding="utf-8") as txt_file:
        text = txt_file.read()

    return text

## 추출된 텍스트 청크 분할
텍스트를 추출한 뒤에는 검색 정확도를 높이기 위해 조금씩 겹치도록 나눠서 작은 단위로 분할(chunk)합니다.

In [ ]:
def chunk_text(text, n, overlap):
    """
    주어진 텍스트를 n자 단위로, 일부가 겹치도록 chunking 합니다.

    Args:
    text (str): 청크할 텍스트입니다.
    n (int): 각 청크의 문자 수입니다.
    overlap (int): 청크 간 겹치는 문자 수입니다.

    Returns:
    List[str]: 청크된 텍스트 리스트입니다.
    """
    chunks = []  # 청크된 텍스트를 저장할 리스트
    
    # overlap만큼 겹치도록 text를 n의 길이로 chunking
    for i in range(0, len(text), n - overlap):
        chunks.append(text[i:i + n])

    return chunks  

## PDF 파일에서 텍스트 추출 및 청크 분할하기
이제 PDF를 로드하고 텍스트를 추출한 후 청크로 분할합니다.

In [ ]:
# PDF 파일의 경로를 정의합니다.
# pdf_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.pdf"

# PDF 파일에서 텍스트 추출
# extracted_text = extract_text_from_pdf(pdf_path)


# text 파일 경로 정의
text_path = "./data_creation/pdf_data/(1) 2024 달라지는 세금제도.txt"

# text 파일 로드
extracted_text = load_text_file(text_path)

# 추출한 텍스트를 1000자씩, 200자가 겹치도록 나눕니다.
text_chunks = chunk_text(extracted_text, 1000, 200)

# 청크된 텍스트의 개수를 출력합니다.
print("Number of text chunks:", len(text_chunks))

# 청크된 텍스트의 첫 번째 조각을 출력합니다.
print("\nFirst text chunk:")
print(text_chunks[0])

## 분할된 청크의 임베딩 생성
임베딩은 텍스트를 숫자 벡터로 변환하여 효율적인 유사도 검색을 가능하게 합니다.

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
def create_embeddings(embedding_model, texts, device='cuda', batch_size=16):
    """
    SentenceTransformer 모델을 사용하여 텍스트의 임베딩 생성

    Args:
        embedding_model: 임베딩을 생성할 SentenceTransformer 모델입니다.
        texts (list): 임베딩을 생성할 입력 텍스트 리스트입니다.
        device (str): 모델을 실행할 장치 ('cuda' for GPU, 'cpu' for CPU).
        batch_size (int): 한번에 처리할 텍스트의 개수

    Returns:
        np.ndarray: 임베딩
    """
    # 모델이 지정된 장치에 있는지 확인합니다.
    embedding_model = embedding_model.to(device)
    
    # 지정된 배치 크기로 임베딩을 생성합니다.
    embeddings = embedding_model.encode(
        texts,
        device=device,
        batch_size=batch_size,  # 메모리 사용량을 줄이기 위해 더 작은 배치 크기를 사용합니다.
        show_progress_bar=True  # 인코딩 진행 상태를 모니터링하기 위해 진행 표시줄 출력
    )
    
    return embeddings

# GPU 사용 가능 여부 확인
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# 모델 로드
model = "BAAI/bge-m3"
embedding_model = SentenceTransformer(model)


# 임베딩 생성
embeddings = create_embeddings(embedding_model, text_chunks, device=device, batch_size=4)

print(f"Generated {len(embeddings)} sentence embeddings.")

## Semantic Search
코사인 유사도를 구현하여 사용자 쿼리에 가장 관련성이 높은 텍스트 청크를 찾습니다.

In [10]:
def cosine_similarity(vec1, vec2):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    Args:
    vec1 (np.ndarray): 첫 번째 벡터입니다.
    vec2 (np.ndarray): 두 번째 벡터입니다.

    Returns:
    float: 두 벡터 간의 코사인 유사도입니다.
    """
    # 두 벡터의 내적을 계산하고 두 벡터의 크기의 곱으로 나눕니다.
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    주어진 쿼리와 임베딩을 이용해 텍스트 chunk에 대해 Semantic Search 수행

    Args:
    query (str): 사용자 쿼리
    text_chunks (List[str]): 검색할 텍스트 청크 리스트
    embeddings (List[dict]): 텍스트 청크에 대한 임베딩 리스트
    k (int): 상위 k개의 관련 텍스트 청크를 반환

    Returns:
    List[str]: 쿼리와 가장 유사한 상위 k개의 텍스트 청크 리스트
    """

    # 쿼리에 대한 임베딩 생성
    query_embedding = create_embeddings(embedding_model, [query])[0]
    similarity_scores = []  # 유사도 점수를 저장할 빈 리스트

    # 쿼리 임베딩과 각 텍스트 청크 임베딩 간의 유사도 점수 계산
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding))
        similarity_scores.append((i, similarity_score))  # 인덱스와 유사도 점수를 리스트에 추가

    # 유사도 점수를 내림차순으로 정렬
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    
    # 상위 k개의 유사한 텍스트 청크의 인덱스를 추출
    top_indices = [index for index, _ in similarity_scores[:k]]
    
    # 상위 k개의 관련 텍스트 청크를 반환
    return [text_chunks[index] for index in top_indices]


## 테스트

In [ ]:
import pandas as pd

# 평가 데이터 로드
df = pd.read_csv('./data_creation/rag_val_new_post.csv')
df.head()

In [ ]:
# 평가 데이터에서 첫 번째 쿼리 추출
query = df['query'][0]

# 시맨틱 검색 수행
top_chunks = semantic_search(query, text_chunks, embeddings, k=3)

# 쿼리 출력
print("Query:", query)

# 쿼리와 연관있는 3개의 chunk 출력
for i, chunk in enumerate(top_chunks):
    print(f"Context {i + 1}:\n{chunk}\n=====================================")

## 검색된 청크를 기반으로 response 생성하기

In [ ]:
def generate_response(system_prompt, user_message ,model_name='gpt-4.1-nano'):
    
    response = client_openai.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},# Define the system prompt for the AI assistant
            {"role": "user", "content": user_message}
        ],
        temperature=0.1,
        top_p=0.9,
        max_tokens=1024,
    )
    # print(response.choices[0].message.content)

    return response

# 사용자 프롬프트
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\n질문: {query}"

# 시스템 프롬프트
system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"

# 응답 생성
ai_response = generate_response(system_prompt, user_prompt)

In [ ]:
print(ai_response.choices[0].message.content)

## 생성 응답 평가하기
생성된 답변과 예상 답변을 비교하여 평가합니다. 답변 평가시에는 LLM을 사용합니다.

In [ ]:
# 평가를 위한 시스템 프롬프트
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# 사용자 쿼리, 생성된 답변, 참고(정답) 답변, 평가 시스템 프롬프트를 결합하여 평가 프롬프트 생성
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {df['generation_gt'][0]}\n{evaluate_system_prompt}"

# 평가 응답 생성
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt,'gpt-4.1-mini')

# 평가 응답 출력
print(evaluation_response.choices[0].message.content)

## 전체 추론 및 평가

In [ ]:
# 생성된 답변과 실제 답변을 비교하여 점수 부여
result = []

for i in range(len(df)):
    query = df['query'][i]
    top_chunks = semantic_search(query, text_chunks, embeddings, k=3)

    # 답변 생성용 시스템 프롬프트
    system_prompt = "당신은 제공된 Context에 기반하여 답변하는 AI 어시스턴트입니다. 답변이 컨텍스트에서 직접 도출될 수 없는 경우, 다음 문장을 사용하세요: '해당 질문에 답변할 충분한 정보가 없습니다.'"
        
    # 답변 생성용 유저 프롬프트
    user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
    user_prompt = f"{user_prompt}\n질문: {query}"

    # response 생성
    ai_response = generate_response(system_prompt, user_prompt, 'gpt-4.1-mini')

    print(ai_response.choices[0].message.content)
    
    # 평가 시스템 프롬프트
    evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

    # 사용자 쿼리, 생성된 답변, 참고(정답) 답변, 평가 시스템 프롬프트를 결합하여 평가 프롬프트 생성
    evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {df['generation_gt'][0]}\n{evaluate_system_prompt}"

    # 평가 응답 생성
    evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt,'gpt-4.1-mini')

    # 평가 응답 출력
    print(evaluation_response.choices[0].message.content)
    result.append(evaluation_response.choices[0].message.content)


## 점수 계산

In [ ]:
import numpy as np
result = [float(x.replace('\n',''))for x in result]
print(np.mean(result))